In [3]:
from lightgbm import LGBMClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, recall_score, f1_score, precision_score
import mlflow
from katonic.ml import MyClient

In [4]:
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client

In [19]:
def metric(actual, pred):
    acc_score = accuracy_score(actual, pred)
    recall = recall_score(actual, pred, average='weighted')
    precision_scr = precision_score(actual, pred, average='weighted')
    f1_scr = f1_score(actual, pred, average='weighted')
    
    return (
        acc_score,
        recall,
        f1_scr,
        precision_scr
    )

In [21]:
mlflow.lightgbm.autolog()
exp_name = "mlflow-test-lightbgm"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)

data = load_iris()
feature_names = [
    name.replace(" ", "_").replace("(", "").replace(")", "")
    for name in data["feature_names"]
]
X_train, X_test, y_train, y_test = train_test_split(
    data["data"], data["target"], test_size=0.2
)

with mlflow.start_run(run_name=exp_name):
    lgb_classifier = LGBMClassifier(
        n_estimators=10,
        max_depth=3,
        learning_rate=1,
        #objective="binary:logistic",
        random_state=123,
        average='weighted'
    )
    lgb_classifier.fit(X_train, y_train)
    y_pred = lgb_classifier.predict(X_test)
    (acc_score, recall, f1_scr, precision_scr) = metric(y_test, y_pred)

    model_metrics = {
        "accuracy_score": acc_score,
        "recall": recall,
        "f1_score": f1_scr,
        "precision_score": precision_scr
    }

    for metric_name, score in model_metrics.items():
        mlflow.log_metric(metric_name, score)

    model_info = mlflow.lightgbm.log_model(lgb_model=lgb_classifier, artifact_path="model")
    

2023/10/25 04:18:09 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


[LightGBM] [Warning] Unknown parameter: average
[LightGBM] [Warning] Unknown parameter: average
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000031 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 89
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score -1.176574
[LightGBM] [Info] Start training from score -1.049822
[LightGBM] [Info] Start training from score -1.073920
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB